# Movie Recommendation

later

In [1]:
# Core
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SKLearn
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

# NLTK
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

# Display
from IPython.core.display import HTML
from movie_display import movie_display

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jeroen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Import movies into a data frame and analyze the availables attributes.

## IMDb JSON

9125 Movies with 18 attributes from IMDb.

Attributes:

- Actors
- ...
- Plot
- Title
- Writer
- ...
- imdbVotes

In [2]:
# Load movies into a dataframe
df = pd.read_json('./dataset/imdbdata.json', orient='columns')
df.head(2)

,Actors,Awards,Country,Director,Genre,Language,Plot,Poster,Production,Rated,Released,Runtime,Title,Writer,Year,imdbId,imdbRating,imdbVotes
0,"Tom Hanks, Tim Allen, Don Rickles, Jim Varney",Nominated for 3 Oscars. Another 23 wins & 18 n...,USA,John Lasseter,"Animation, Adventure, Comedy",English,A cowboy doll is profoundly threatened and jea...,https://images-na.ssl-images-amazon.com/images...,Buena Vista,G,22 Nov 1995,81 min,Toy Story,"John Lasseter (original story by), Pete Docter...",1995,114709,8.3,"666,855"
1,"Robin Williams, Jonathan Hyde, Kirsten Dunst, ...",4 wins & 9 nominations.,USA,Joe Johnston,"Action, Adventure, Family","English, French",When two kids find and play a magical board ga...,https://images-na.ssl-images-amazon.com/images...,Sony Pictures Home Entertainment,PG,15 Dec 1995,104 min,Jumanji,"Jonathan Hensleigh (screenplay), Greg Taylor (...",1995,113497,6.9,"223,000"


### Feature Extraction

Pipeline: https://stackoverflow.com/questions/36182502/add-stemming-support-to-countvectorizer-sklearn

Excluding stopwords and stemming words according to the NLTK English language dictionaries.

#### Stopwords

Frequent words with no meaning. Like: 'as', 'you', 'the'.

#### Stemming

Useful but also transforms correct words into incorrect ones. Like `james` to `jame` and `territory` to `terri`.

In [3]:
features = df.copy()

# Get the stopwords for the English language
useless_words = stopwords.words('english')
# Create a stemmer for the English language
stemmer = SnowballStemmer("english", ignore_stopwords=True)

print(useless_words[:10])

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [20]:
plots = []
for sentence in features.Plot:
    words = sentence.lower().split()
    words = [stemmer.stem(w) for w in words if w not in useless_words]
    plots.append(' '.join(words))

features['PlotStripped'] = plots
plots[0]

'cowboy doll profound threaten jealous new spaceman figur supplant top toy boy room.'

##### Count Vectorizer
Finding the frequencies of words in the plots using a bag of words approach and transforming the term count to term frequency.

In [5]:
vectorizer = CountVectorizer()
bag_of_words = vectorizer.fit(plots).transform(plots)

tfidf_transformer = TfidfTransformer()
tfidf_transformed = tfidf_transformer.fit_transform(bag_of_words)
print(tfidf_transformed)

  (0, 15753)	0.29240981532129123
  (0, 15689)	0.24046800457052322
  (0, 15544)	0.22150792358457966
  (0, 15067)	0.35904030100261597
  (0, 14508)	0.375179319771497
  (0, 13292)	0.2549551797587505
  (0, 12250)	0.3111177071772179
  (0, 10725)	0.14056017234989235
  (0, 8268)	0.2815858424310979
  (0, 5894)	0.2730847991986979
  (0, 4702)	0.30067463401543315
  (0, 3699)	0.28556953617885883
  (0, 2150)	0.1748193225642691
  (1, 16044)	0.14745135162289927
  (1, 15823)	0.2504788883525001
  (1, 14820)	0.2075814399033663
  (1, 12845)	0.25473380937430284
  (1, 11864)	0.21438959349206538
  (1, 9557)	0.14122901300077814
  (1, 9471)	0.24481254824217555
  (1, 8596)	0.24142117814367708
  (1, 7485)	0.2677674364113207
  (1, 6389)	0.4755675137495493
  (1, 5926)	0.3227679538002733
  (1, 5920)	0.13882858482652902
  :	:
  (9123, 1460)	0.16603863332710772
  (9123, 1457)	0.12284293783002358
  (9123, 1002)	0.1814252505152987
  (9123, 788)	0.19557276702605814
  (9123, 738)	0.12790986819694314
  (9123, 483)	0.20923

##### Tf-Idf
Finding the frequencies of words in the plots directly using the Tf-Idf vectorizer.

In [6]:
vectorizer = TfidfVectorizer()
tfidf = vectorizer.fit_transform(plots)

print("  (movie_index, word_index)\tfrequency\n")
print(tfidf)

  (movie_index, word_index)	frequency

  (0, 3699)	0.28556953617885883
  (0, 4702)	0.30067463401543315
  (0, 12250)	0.3111177071772179
  (0, 15544)	0.22150792358457966
  (0, 8268)	0.2815858424310979
  (0, 10725)	0.14056017234989235
  (0, 14508)	0.375179319771497
  (0, 5894)	0.2730847991986979
  (0, 15067)	0.35904030100261597
  (0, 15689)	0.24046800457052322
  (0, 15753)	0.29240981532129123
  (0, 2150)	0.1748193225642691
  (0, 13292)	0.2549551797587505
  (1, 16044)	0.14745135162289927
  (1, 8596)	0.24142117814367708
  (1, 5920)	0.13882858482652902
  (1, 11864)	0.21438959349206538
  (1, 9471)	0.24481254824217555
  (1, 1993)	0.26481534152638636
  (1, 6389)	0.4755675137495493
  (1, 12845)	0.25473380937430284
  (1, 9557)	0.14122901300077814
  (1, 15823)	0.2504788883525001
  (1, 4114)	0.28844770483087717
  (1, 7485)	0.2677674364113207
  :	:
  (9123, 481)	0.1735077100896544
  (9123, 6402)	0.18484711756795755
  (9123, 1002)	0.1814252505152987
  (9123, 2969)	0.4184686576483837
  (9123, 5323)	0.

## Feature vectors

In [24]:
class Recommender():

    def __init__(self, n_recommendations=5):
        self.nbrs = NearestNeighbors(n_neighbors=n_recommendations + 1, algorithm='auto', metric='cosine')
    
    def fit(self, features):
        self.nbrs.fit(features)
        self.distances, self.graph = self.nbrs.kneighbors(features)
    
    def recommend(self, movie_index):
        # We don't want to include the same movie so we exclude it. 
        return self.graph[movie_index, 1:]

recommender = Recommender(n_recommendations=7)
recommender.fit(tfidf)

## Recommend movies

In [36]:
movie = 1

recommendation = recommender.recommend(movie)

print('Title:', df.iloc[movie].Title)
print('Recommendations:')
display(HTML(movie_display.show([df.iloc[i] for i in recommendation])))

Title: Jumanji
Recommendations:
